**Mufti Alfarokhul Azam <br> 510317**

In [36]:
import mysql.connector
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [37]:
db = mysql.connector.connect(
  host="localhost",
  user="muftialroaz",
  password="password",
  database="rekmed",
  port=3306
)

Rekam Medis

In [38]:
# Retrieve all records from the 'customers' table
cursor = db.cursor()
cursor.execute("SELECT rekam_medis.rm_id, rm_diagnosis.nama_diagnosis, dokter.user_id AS dokter_user_id, dokter.nama AS dokter_nama, user.username AS dokter_username, pasien.mr AS pasien_mr, pasien.nama AS pasien_nama FROM rekam_medis LEFT JOIN dokter ON rekam_medis.user_id = dokter.user_id LEFT JOIN rm_diagnosis ON rekam_medis.rm_id = rm_diagnosis.rm_id LEFT JOIN pasien ON rekam_medis.mr = pasien.mr LEFT JOIN user ON dokter.user_id = user.id;")

# Fetch all results as a list of tuples
rekam_medis = cursor.fetchall()
rekam_medis = pd.DataFrame(rekam_medis)
rekam_medis.rename(columns={0: 'rm_id', 1: 'diagnosis_nama', 2:"dokter_user_id", 3: 'dokter_nama', 4: "dokter_username", 5: "pasien_mr", 6:"pasien_nama"}, inplace=True)

cursor.close()
db.close()

In [39]:
from pasien import pasien_data
from diagnosis import diagnosis_data
from dokter import dokter_data
from modules import preprocess_input
import db

input_data = input()
search = preprocess_input(input_data)
nama_pasien = pasien_data(search)
diagnosis = diagnosis_data(search)
nama_dokter = dokter_data(search)
rekam_medis = db.rekam_medis.copy()

In [40]:
rekam_medis

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa
0,1,influenza,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-10
1,2,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-10
2,3,Kosong,112,Yudhistya,huroncom@gmail.com,108000003,Ahmad Surya,2016-08-10
3,4,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11
4,5,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11
...,...,...,...,...,...,...,...,...
284,285,Kosong,114,dr Yudhistya SpOG,indosehat,109000071,Noah Lee,2016-11-30
285,286,Kosong,114,dr Yudhistya SpOG,indosehat,109000070,Ava Santoso,2016-11-30
286,287,Kosong,114,dr Yudhistya SpOG,indosehat,109000069,Jack Wilson,2016-11-30
287,288,Kosong,114,dr Yudhistya SpOG,indosehat,109000105,Aaliyah Davis,2016-11-30


In [41]:
# rekam_medis['cosine_diagnosis'] = diagnosis['cosine']
# rekam_medis['cosine_dokter'] = nama_dokter['cosine']
# rekam_medis['cosine_pasien'] = nama_pasien['cosine']
# rekam_medis['cosine'] = (nama_pasien['cosine'] + diagnosis['cosine'] + nama_dokter['cosine'])/3
# Menggabungkan konten dari tiga kolom ke satu kolom baru 'combined'
rekam_medis['combined_text'] = rekam_medis['pasien_nama'] + ' ' + rekam_medis['dokter_nama'] + ' ' + rekam_medis['diagnosis_nama']
# rekam_medis.sort_values(by='cosine_diagnosis', ascending=False)

# diagnosis_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'diagnosis_nama']
# pasien_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'pasien_nama']
# dokter_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'dokter_nama']

In [42]:
import pandas as pd
from modules import preprocess, vektorisasi, cosine_sim, bm_25

doc_pre = preprocess(rekam_medis['combined_text'])

doc_vec, search_vec = vektorisasi(doc_pre, search)
cosi = cosine_sim(doc_vec, search_vec)

  # gabungkan dalam satu dataframe
df_vector = pd.concat([rekam_medis, cosi], axis=1) 
df_vector.sort_values(by='cosine', ascending=False)

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa,combined_text,cosine
0,1,influenza,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-10,Sri Wahyuni Yudhistya influenza,0.866025
76,77,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-17,Sri Wahyuni Yudhistya Kosong,0.866025
114,114,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-10-01,Sri Wahyuni Yudhistya Kosong,0.866025
18,18,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-12,Sri Wahyuni Yudhistya Kosong,0.866025
5,6,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-11,Sri Wahyuni Yudhistya Kosong,0.866025
...,...,...,...,...,...,...,...,...,...,...
154,155,ispa,93,anita,saskyagitaamira@yahoo.com,94000018,Maya Prasetyo,2016-10-27,Maya Prasetyo anita ispa,0.000000
155,156,Diabetes mellitus,93,anita,saskyagitaamira@yahoo.com,94000019,Dian Wulandari,2016-10-27,Dian Wulandari anita Diabetes mellitus,0.000000
156,157,Kosong,93,anita,saskyagitaamira@yahoo.com,94000020,Bambang Pratomo,2016-10-27,Bambang Pratomo anita Kosong,0.000000
157,158,Kosong,93,anita,saskyagitaamira@yahoo.com,94000021,Ayu Sari,2016-10-27,Ayu Sari anita Kosong,0.000000


In [43]:
import pandas as pd

# Contoh data hasil pencarian
data = {
    'cosine': df_vector['cosine'].tolist(),
}

df = pd.DataFrame(data)

def evaluate_top_k(df, column_name, k):
    # Urutkan berdasarkan nilai cosine similarity dari tertinggi ke terendah
    df_sorted = df.sort_values(by=column_name, ascending=False)
    
    # Ambil top-k hasil
    top_k_results = df_sorted.head(k)
    
    # Hitung metrik evaluasi
    relevant_count = top_k_results[top_k_results[column_name] > 0].shape[0]
    precision = relevant_count / k
    recall = relevant_count / df[df[column_name] > 0].shape[0] if df[df[column_name] > 0].shape[0] > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1


print("Evaluasi dengan keyword:", input_data)

# Evaluasi untuk kolom cosine_pasien
precision_pasien, recall_pasien, f1_pasien = evaluate_top_k(df, 'cosine', 20)
print("\nCosine Pasien:")
print("Precision:", precision_pasien)
print("Recall:", recall_pasien)
print("F1-Score:", f1_pasien)


Evaluasi dengan keyword: Sri Wahyuni YudhiStya

Cosine Pasien:
Precision: 1.0
Recall: 0.15503875968992248
F1-Score: 0.2684563758389262


In [44]:
import pandas as pd

# Contoh data hasil pencarian
data = {
    'cosine': df_vector['cosine'].tolist(),
}

df = pd.DataFrame(data)

def evaluate_top_k(df, column_name, k):
    # Urutkan berdasarkan nilai cosine similarity dari tertinggi ke terendah
    df_sorted = df.sort_values(by=column_name, ascending=False)
    
    # Ambil top-k hasil
    top_k_results = df_sorted.head(k)
    
    # Hitung metrik evaluasi
    precision = top_k_results[top_k_results[column_name] > 0].shape[0] / k
    recall = top_k_results[top_k_results[column_name] > 0].shape[0] / df.shape[0] if df.shape[0] > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

print("Evaluasi dengan keyword:", input_data)
# Evaluasi untuk kolom cosine_diagnosis
precision_diag, recall_diag, f1_diag = evaluate_top_k(df, 'cosine', 20)
print("Cosine Diagnosis:")
print("Precision:", precision_diag)
print("Recall:", recall_diag)
print("F1-Score:", f1_diag)


Evaluasi dengan keyword: Sri Wahyuni YudhiStya
Cosine Diagnosis:
Precision: 1.0
Recall: 0.06920415224913495
F1-Score: 0.12944983818770228


In [45]:
df_vector.sort_values(by='cosine', ascending=False).head(20)

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa,combined_text,cosine
0,1,influenza,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-10,Sri Wahyuni Yudhistya influenza,0.866025
76,77,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-17,Sri Wahyuni Yudhistya Kosong,0.866025
114,114,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-10-01,Sri Wahyuni Yudhistya Kosong,0.866025
18,18,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-12,Sri Wahyuni Yudhistya Kosong,0.866025
5,6,Kosong,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-11,Sri Wahyuni Yudhistya Kosong,0.866025
251,252,Kosong,111,dr_lukman,dr_lukman,108000001,Sri Wahyuni,2016-10-11,Sri Wahyuni dr_lukman Kosong,0.516398
21,21,Prolonged second stage (of labour),114,dr Yudhistya SpOG,indosehat,109000008,Sri Utami,2016-08-17,Sri Utami dr Yudhistya SpOG Prolonged second s...,0.365148
24,24,Delivery by emergency caesarean section,114,dr Yudhistya SpOG,indosehat,109000008,Sri Utami,2016-08-18,Sri Utami dr Yudhistya SpOG Delivery by emerge...,0.365148
81,82,Kosong,112,Yudhistya,huroncom@gmail.com,108000007,Dodi Prasetyo,2016-09-06,Dodi Prasetyo Yudhistya Kosong,0.288675
252,253,Kosong,112,Yudhistya,huroncom@gmail.com,108000011,Rendra Pratama,2016-11-13,Rendra Pratama Yudhistya Kosong,0.288675


In [130]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Contoh DataFrame
data = {'column1': ['This is document 1.', 'Another document here.', 'Document with some keywords.'],
        'column2': ['Document with similar keywords.', 'Some text in document 2.', 'More content in document.'],
        'column3': ['Text in the third document.', 'Document with common words.', 'Additional content.']}

df = pd.DataFrame(data)

# Kata kunci (keyword) yang akan dibandingkan
keyword = "keywords"

# Menggabungkan konten dari tiga kolom ke satu kolom baru 'combined'
df['combined'] = df['column1'] + ' ' + df['column2'] + ' ' + df['column3']

# Menghitung vektor TF-IDF untuk kata kunci dan dokumen-dokumen
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([keyword] + list(df['combined']))

# Menghitung kemiripan kosinus
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# Menambahkan nilai kemiripan kosinus ke DataFrame
df['cosine_similarity'] = cosine_similarities

# Menampilkan DataFrame dengan nilai kemiripan kosinus
print(df[['combined', 'cosine_similarity']])


                                            combined  cosine_similarity
0  This is document 1. Document with similar keyw...           0.196881
1  Another document here. Some text in document 2...           0.000000
2  Document with some keywords. More content in d...           0.207377


In [49]:
import numpy as np

# Assuming true relevance labels for the top-K documents
true_relevance_labels = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]  # Example labels, adjust as needed

# Cosine similarity scores for documents 1 to 6
cosine_similarity_scores = [0.888675, 0.866025, 0.788675, 0.656025, 0.588675, 0.466025]

# Sort documents based on cosine similarity scores
sorted_indices = np.argsort(cosine_similarity_scores)[::-1]

# Retrieve top-K documents
top_k_indices = sorted_indices[:10]

# Calculate precision@K and recall@K
relevant_at_k = np.sum(np.array(true_relevance_labels)[top_k_indices])
precision_at_k = relevant_at_k / 10
recall_at_k = relevant_at_k / np.sum(true_relevance_labels)

# Calculate average precision
avg_precision = np.mean([np.sum(true_relevance_labels[:i+1]) / (i+1) for i in range(10) if true_relevance_labels[i]])

# Print the results
print(f'Precision@K: {precision_at_k:.4f}')
print(f'Recall@K: {recall_at_k:.4f}')
print(f'Average Precision: {avg_precision:.4f}')


Precision@K: 0.3000
Recall@K: 1.0000
Average Precision: 1.0000


In [65]:
def precision_at_k(true_relevance_labels, k):
    relevant_at_k = sum(true_relevance_labels[:k])
    return relevant_at_k / k if k > 0 else 0

def recall_at_k(true_relevance_labels, k):
    relevant_at_k = sum(true_relevance_labels[:k])
    total_relevant = sum(true_relevance_labels)
    return relevant_at_k / total_relevant if total_relevant > 0 else 0

def average_precision(true_relevance_labels):
    avg_precision = 0
    num_relevant = 0

    for i, relevance in enumerate(true_relevance_labels):
        if relevance == 1:
            num_relevant += 1
            avg_precision += num_relevant / (i + 1)

    return avg_precision / num_relevant if num_relevant > 0 else 0

# Example usage:
true_relevance_labels = [1, 0, 1, 0, 1, 0, 0, 1, 0, 1]  # Replace with your actual relevance labels
k_value = 10  # Replace with your desired K value

precision_k = precision_at_k(true_relevance_labels, k_value)
recall_k = recall_at_k(true_relevance_labels, k_value)
avg_precision_value = average_precision(true_relevance_labels)

# Print the results
print(f'Precision@{k_value}: {precision_k:.4f}')
print(f'Recall@{k_value}: {recall_k:.4f}')
print(f'Average Precision: {avg_precision_value:.4f}')

Precision@10: 0.5000
Recall@10: 1.0000
Average Precision: 0.6533


In [62]:
def precision_at_k(true_labels, predicted_labels, k):
    relevant_at_k = sum(true_labels[i] for i in predicted_labels[:k])
    return relevant_at_k / k if k > 0 else 0

def recall_at_k(true_labels, predicted_labels, k):
    relevant_at_k = sum(true_labels[i] for i in predicted_labels[:k])
    total_relevant = sum(true_labels)
    return relevant_at_k / total_relevant if total_relevant > 0 else 0

def average_precision(true_labels, predicted_labels):
    precision_values = []
    num_relevant = 0

    for i, label in enumerate(predicted_labels):
        if true_labels[label] == 1:
            num_relevant += 1
            precision_at_i = num_relevant / (i + 1)
            precision_values.append(precision_at_i)

    if not precision_values:
        return 0

    return sum(precision_values) / num_relevant

# Example usage:
true_relevance_labels = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
predicted_ranking = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  # Adjust as needed

k = 286  # Choose your desired value for K

precision_k = precision_at_k(true_relevance_labels, predicted_ranking, k)
recall_k = recall_at_k(true_relevance_labels, predicted_ranking, k)
avg_precision = average_precision(true_relevance_labels, predicted_ranking)

# Print the results
print(f'Precision@{k}: {precision_k:.4f}')
print(f'Recall@{k}: {recall_k:.4f}')
print(f'Average Precision: {avg_precision:.4f}')


Precision@286: 0.0105
Recall@286: 1.0000
Average Precision: 0.7556


In [50]:
from rank_bm25 import BM25Okapi
import pandas as pd

# Contoh data dokumen
documents = [
    "Dokumen pertama adalah contoh dokumen",
    "Ini adalah contoh dokumen kedua",
    "Contoh yang lebih panjang untuk dokumen ketiga",
    "Ini adalah dokumen contoh terakhir"
]

# Pembentukan indeks BM25
tokenized_documents = [doc.split() for doc in documents]
bm25 = BM25Okapi(tokenized_documents)

# Contoh kueri pencarian
query = "contoh dokumen panjang"

# Hitung relevansi BM25
scores = bm25.get_scores(query.split())

# Urutkan dokumen berdasarkan skor relevansi
ranked_documents = sorted(zip(scores, documents), key=lambda x: x[0], reverse=True)

ranked = pd.DataFrame(scores)
ranked = ranked.rename(columns={0:'ranking'})
ranked
# # Tampilkan hasil
# print("Hasil Pencarian:")
# for score, doc in ranked_documents:
#     print(f"Skor: {score:.4f}, Dokumen: {doc}")

,ranking
0,0.170592
1,0.170592
2,0.827542
3,0.170592
